In [85]:
# pip install transformers datasets
import torch
print("torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
classifier = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    top_k=None,     # replaces return_all_scores=True
    device=0        # use GPU 0
)
tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")


torch: 2.8.0+cpu
CUDA available: False


Device set to use cpu


In [86]:
import json

# Read files
with open("../data/DocumentForParsing.txt", "r", encoding="utf-8") as f:
    data1 = json.load(f)

dronesText = data1["text"]

with open("../data/DocumentForParsing2.txt", "r", encoding="utf-8") as f:
    data2 = json.load(f)

powerPlantAttackText = data2["reference_text"]

print(dronesText[:500])
print("\n")
print(powerPlantAttackText[:500])


As tensions between Donovia and the Western Bloc reach a boiling point, reports of mysterious drone sightings in Temple, Texas, have left residents on edge and sparked concerns about foreign interference in domestic affairs. Central Texas City Grapples with Mysterious Sightings Amidst Growing Tensions The Donovian government has issued a statement denying any involvement, but insiders claim otherwise. The situation is further complicated by the escalating global power struggle between Donovia an


Killeen Cyber Attack Disrupts Local Power Plant. KILLEEN, TX – On the morning of March 26th, the Killeen Power Plant experienced a cyber-attack that cut power to about 2,300 homes in Killeen. Operators detected unusual network activity at 4:00 a.m. and shut off four feeder circuits as a precaution. Workers are facilitating return of power and expect power to return at around 10:00am. What Happened Systems Hit: Main control server and backup communication lines. Impact: Widespread outages in f

In [87]:
def avg_emotion_scores(text, tokenizer, classifier, chunk_size=None):
    """
    Split `text` into token-sized chunks, run `classifier` on each chunk,
    and return a dict of averaged label scores across all chunks.
    """
    # choose chunk size (leave room for special tokens)
    if chunk_size is None:
        model_max = getattr(tokenizer, "model_max_length", 512)
        chunk_size = max(2, model_max - 2)

    # encode without special tokens then split into chunks of token ids
    input_ids = tokenizer.encode(text, add_special_tokens=False)
    if len(input_ids) == 0:
        return {}

    token_chunks = [input_ids[i : i + chunk_size] for i in range(0, len(input_ids), chunk_size)]
    # decode token chunks back to strings for the pipeline
    text_chunks = [
        tokenizer.decode(ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        for ids in token_chunks
    ]

    # classify all chunks at once; ensure truncation for safety
    chunk_results = classifier(text_chunks, truncation=True, max_length=chunk_size)

    # aggregate scores per label
    sums = {}
    counts = {}
    for chunk_out in chunk_results:            # chunk_out is a list of {'label','score'} dicts
        for item in chunk_out:
            lbl = item["label"]
            sc = float(item["score"])
            sums[lbl] = sums.get(lbl, 0.0) + sc
            counts[lbl] = counts.get(lbl, 0) + 1

    # compute averages
    avg_scores = {lbl: (sums[lbl] / counts[lbl]) for lbl in sums}
    return avg_scores


In [88]:
def convertToDict(results):
    return {item["label"]: float(item["score"]) for item in results[0]}


In [89]:
dronesResults = avg_emotion_scores(dronesText, tokenizer, classifier)
print(dronesResults)

powerPlantAttackResults = convertToDict(classifier(powerPlantAttackText))
print(powerPlantAttackResults["fear"])
print("\n")

Token indices sequence length is longer than the specified maximum sequence length for this model (1127 > 512). Running this sequence through the model will result in indexing errors


{'fear': 0.8270518183708191, 'anger': 0.08275649261971314, 'neutral': 0.0638207762191693, 'disgust': 0.010619757192519804, 'sadness': 0.008104392948249975, 'surprise': 0.005372426627824704, 'joy': 0.0022743274845803776}
0.6234105825424194




In [ ]:
from typing import Dict

def clamp(x: float) -> float:
    return max(0.0, min(1.0, x))

def emotions_to_fsmt(emotions: Dict[str, float], intensity: float = 0.5) -> Dict[str, float]:
    """
    Convert emotion probabilities (0-1) to:
      - fear
      - stress
      - morale
      - trust
    All outputs are clamped to [0,1].
    Expected emotion keys: 'fear','anger','neutral','disgust','sadness','surprise','joy'.
    """
    # ensure expected keys exist
    keys = ['fear','anger','neutral','disgust','sadness','surprise','joy']
    e = {k: float(emotions.get(k, 0.0)) for k in keys}
    intensity = float(clamp(intensity))

    # Fear: mostly the 'fear' signal, slightly boosted by surprise and anger when arousal is higher
    fear = e['fear'] * (0.6 + 0.4 * intensity) + 0.2 * e['surprise'] + 0.1 * e['anger']

    # Stress: combination of fear, anger and surprise; scaled by intensity and reduced by neutrality
    stress_base = 0.6 * e['fear'] + 0.25 * e['anger'] + 0.15 * e['surprise']
    stress = stress_base * (0.5 + 0.5 * intensity) + 0.1 * (1 - e['neutral'])

    # Morale: increases with joy and neutral, decreases with negative emotions
    negative = e['fear'] + e['anger'] + e['sadness'] + e['disgust']
    positive = e['joy'] + 0.7 * e['neutral']
    morale = (positive - 0.9 * negative) * (0.6 + 0.4 * (1 - intensity)) + 0.1

    # Trust: boosted by neutral & joy, heavily reduced by fear/anger
    trust = 0.5 * e['neutral'] + 0.6 * e['joy'] - 0.9 * (e['fear'] + e['anger']) + 0.2 * intensity

    # clamp outputs to [0,1]
    return {
        'fear': clamp(fear),
        'stress': clamp(stress),
        'morale': clamp(morale),
        'trust': clamp(trust)
    }

In [92]:
print(emotions_to_fsmt(dronesResults))
print(emotions_to_fsmt(powerPlantAttackResults))


{'fear': 0.6709915892841917, 'stress': 0.4819124810067782, 'morale': 0.0, 'trust': 0.0}
{'fear': 0.506616160646081, 'stress': 0.3625632446724921, 'morale': 0.0, 'trust': 0.0}
